In [ ]:
%matplotlib inline

import os
import pandas as pd
import lasio as ls
#import selenium
#import phantomjs
import matplotlib.pyplot as plt
import numpy as np
import numpy.lib.recfunctions as rfn
#import re
#from azure.datalake.store import core, lib
#import tempfile
import sys
import hvplot.pandas
import holoviews as hv
from holoviews import opts, streams
from holoviews.plotting.links import DataLink
hv.extension('bokeh', logo=None)
from IPython.display import display, HTML
from bokeh.models.tools import LassoSelectTool, BoxSelectTool  # tools=['hover']
box_select = BoxSelectTool()
lasso_select = LassoSelectTool()
import panel as pn
pn.extension()
from IPython.display import display, HTML

client_secret = "Bw2w*31.CmbIiP.i2EILQ=HN@xr]yeu?"

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 80%; }
    div#maintoolbar-container { width: 80%; }
</style>
"""))

## Import real well logs to use as basis for fake well logs

In [ ]:
filepath = '../Documentation/Well log LAS files/WKF W8A LWD VISION RM 0_5m Drilling.LAS'

In [ ]:
def las_to_df(filepath):
    las = ls.read(filepath)
    curve_df = las.df()
    curve_df = curve_df.replace(-999.25,np.nan)
    curve_df.reset_index(inplace=True)
    curve_df.rename(columns={curve_df.columns[0]:'Depth'}, inplace=True)
    las.curves[0].mnemonic = 'Depth'
    return curve_df, las

In [ ]:
df_realwell, las_realwell = las_to_df(filepath)

In [ ]:
# Create new df only containing relevant curves (9) within a start and stop depth:
def get_curves(df, curvenames, start_depth, stop_depth):
    df_new = df.loc[start_depth<=df['Depth']].loc[df['Depth']<stop_depth][curvenames]
    return df_new

In [ ]:
all_curves = ['Depth', 'GR_ARC', 'A16H', 'A28H', 'A40H', 'TNPH', 'ROP5_RM', 'RHOB', 'DCAV'] # Relevant curve names for the chosen well

start_depth = df_realwell['Depth'].iloc[0]
end_depth = df_realwell['Depth'].iloc[-1]
step_depth = df_realwell['Depth'].iloc[1] - df_realwell['Depth'].iloc[0]

df_curves = get_curves(df_realwell, all_curves, start_depth, end_depth) # New df containing specific curves and start and stop depths

In [ ]:
start_depth, end_depth, step_depth # Values to be used as input parameters

## Set input parameters

In [ ]:
Number_of_files = int(input("Number of files to generate: "))
depth_start = int(input("Depth reading start: "))
depth_end = int(input("Depth reading end: "))
depth_step = float(input("number of recordings per meter : ")) # 1=1 per m, 0.5=2 per m, 12= every 12 m
wellname = input('name of wells: ')
filename = input('name of files: ')

In [ ]:
len(np.arange(depth_start, depth_end, depth_step))

## Create LAS-file from scratch

In [ ]:
def file_gen_input(wellname):
    file_arr = np.arange(1,Number_of_files+1,1)
    lasfile = ls.LASFile()
    from datetime import datetime
    lasfile.well.DATE = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    lasfile.well.WELL = wellname
    lasfile.params['CR8TOR'] = ls.HeaderItem('CR8TOR', value='Sebastian Aegidius')
    lasfile.other = 'LAS file creator'
    depth = np.arange(depth_start, depth_end, depth_step)
    lasfile.add_curve('DEPTH (m)', depth, unit = 'm')
    return lasfile, depth

In [ ]:
lasfile, depth = file_gen_input(wellname)

In [ ]:
def add_data(names, info_eq, fgi): # (Name of curve info, equation to produce data, file_gen_input_curve)
    data_eq = info_eq
    #fake[:3] = np.nan # Set first three values to NaN
    fgi.add_curve(names[0], data_eq, unit=names[1], descr=names[2]) # (parameter-name, datavalues, unit, description)
    return

## Add multiple files, equations and data

In [ ]:
def multiple_files(Number_of_files, wellname = wellname):
    list_of_all_wells = []
    for i in range(Number_of_files):
        sample = file_gen_input(wellname+'_'+str(i))
        list_of_all_wells.append(sample[0])
    return list_of_all_wells

In [ ]:
params = [['GR', 'api', 'Gamma Ray'], 
          ['RD', 'ohm.m', 'Deep Resistivity'], 
          ['RM', 'ohm.m', 'Medium Resistivity'], 
          ['RS', 'ohm.m', 'Shallow Resistivity'], 
          ['NPHI', 'v/v', 'Neutron Porosity'], 
          ['RHOB', 'g/cm3', 'Density'], 
          ['DT', 'ms/ft', 'Sonic Velocity'], 
          ['CALI', 'inch', 'Caliper'], 
          ['BS', 'inch', 'Bit Size']
         ]

def eq_realdata(df):
    equation = [df[df.columns[0]].values, 
                df[df.columns[1]].values, 
                df[df.columns[2]].values,
                df[df.columns[3]].values, 
                df[df.columns[4]].values,
                df[df.columns[5]].values,
                df[df.columns[6]].values, 
                df[df.columns[7]].values, 
                df[df.columns[8]].values]
    return equation

In [ ]:
def eq(lasfiles_object, eq_list):
    equations = []
    for i in range(len(lasfiles_object)): # Make unique equations for each las-file
        equations.append(eq_list)
    return equations

In [ ]:
def add_fakedata_multiple_files(fgi, equations, param_list):
    for las in range(len(fgi)): # Go through each las-file
        random = np.abs(np.random.gamma(shape=3, scale=2, size=len(depth))) # Random-data MUST be inside for-loop!
        random_shift = 10 * np.random.sample() - 5
        [add_data([param_list[i][0], param_list[i][1], param_list[i][2]], 
                      abs(equations[las][i]*random + equations[las][i]*random_shift), fgi[las]) for i in range(len(param_list))] # add 'random' data to a las-file
    return

In [ ]:
def write_to_las(fgi, filename):
    [fgi[i].write(filename+'_'+str(i)+'.las', version=2) for i in range(len(fgi))];
    return

In [ ]:
list_of_all_wells = multiple_files(Number_of_files) # X number of las-files

In [ ]:
add_fakedata_multiple_files(list_of_all_wells, eq(list_of_all_wells, eq_realdata(df_curves)), params) # Include some data in each file

In [ ]:
write_to_las(list_of_all_wells, filename) # Update las-files with the new data. TAKES ABOUT 20-30 SEC FOR 100 LAS-FILES.

## Convert las to dataframe

In [ ]:
def las_to_df(lasfile):
    df = lasfile.df()
    df.reset_index(inplace=True)
    df['WELLNAME'] = lasfile.well.WELL.value
    return df

In [ ]:
list_of_wells_df = [las_to_df(list_of_all_wells[i]) for i in range(len(list_of_all_wells))]

In [ ]:
list_of_wells_df

## Plotting

In [ ]:
def get_numerics(df, depthcurvename): # only return the data that is numeric
    curve_list=list(df.columns[(df.dtypes.values == np.dtype('float64'))])
    curve_list.remove(depthcurvename) # Get rid of depth, so we have 9 remaining logs to plot
    return curve_list;

In [ ]:
num_curves = [get_numerics(list_of_wells_df[i], 'DEPTH (m)') for i in range(len(list_of_wells_df))]

In [ ]:
def curve_plot_numerics(log, df, depthname, group=None, alpha=1, colors='blue', height=600, width=300):
    aplot = df.hvplot(x=depthname, y=log, by='WELLNAME', groupby=group ,invert=True, flip_yaxis=True, shared_axes=True, alpha=alpha, color=colors,
                       height=height, width=width).opts(fontsize={'labels': 16,'xticks': 14, 'yticks': 14})
    return aplot;

In [ ]:
# Plot 1/4 of the wells:
plotlists = [curve_plot_numerics(x, list_of_wells_df[i], 'DEPTH (m)') for i in range(int(len(list_of_wells_df)/4)) for x in num_curves[i]]

In [ ]:
hv.Layout(plotlists).cols(len(num_curves[0])) # Each num_curves[i] contains 9 well logs, so all well logs from same well will be plotted on same row

## Grouped plotting

In [ ]:
multiple_wells_df = pd.concat(list_of_wells_df, ignore_index=True) # [:5] -> 5 first wells

In [ ]:
num_curves2 = get_numerics(multiple_wells_df, 'DEPTH (m)')
len(num_curves)

In [ ]:
grouped_plot = [curve_plot_numerics(x, multiple_wells_df, 'DEPTH (m)', group='WELLNAME') for x in num_curves2]

In [ ]:
hv.Layout(grouped_plot).cols(len(num_curves2))

## Overlay plotting

In [ ]:
# Goal: interactively choose what wells to show!
from ipywidgets import interact
import ipywidgets as widgets
unique_wells = list(multiple_wells_df['WELLNAME'].unique())
@interact(wellA = widgets.Dropdown(
                    options = unique_wells,
                    value = unique_wells[0], # Set first well as default value
                    description = 'Well A:',
                    disabled = False),
          wellB = widgets.Dropdown(
                    options = unique_wells,
                    value = unique_wells[1], # Set second wel as default value
                    description = 'Well B:',
                    disabled = False),
          log = widgets.Dropdown(
                    options = num_curves[0], # all num_curves[i] contains same 9 parameters
                    value = num_curves[0][0], # choose first parameter (GR) as default value
                    description = 'Log param:',
                    disabled = False)
)

def overlay_plots(log, wellA, wellB, colorA='darkblue', colorB='gold'):
    WellA = multiple_wells_df.loc[multiple_wells_df['WELLNAME'] == wellA]
    WellB = multiple_wells_df.loc[multiple_wells_df['WELLNAME'] == wellB]
    overlays = hv.Overlay(curve_plot_numerics(log, WellA, 'DEPTH (m)', colors=colorA, alpha=0.5, height=800, width=400) * 
                          curve_plot_numerics(log, WellB, 'DEPTH (m)', colors=colorB, alpha=0.5, height=800, width=400))
    hv.output(overlays, widget_location='left')
    return

In [ ]:
def overlay_plots(multi_df, wellA, wellB, colorA='darkblue', colorB='gold'): 
    WellA = multi_df.loc[multi_df['WELLNAME'] == wellA]
    WellB = multi_df.loc[multi_df['WELLNAME'] == wellB]
    overlays = hv.Layout([hv.Overlay(curve_plot_numerics(x, WellA, 'DEPTH (m)', colors=colorA, alpha=0.5) * 
                          curve_plot_numerics(x, WellB, 'DEPTH (m)', colors=colorB, alpha=0.5))
               for x in num_curves2]).cols(len(num_curves2))
    return overlays

overlay_plots(multiple_wells_df, 'RandGamma_1', 'RandGamma_3') # Overlay plot of las-file 1 and las-file 3

## Add some header items (location, ...)

In [ ]:
def lat_long(fgi, fgi_df):
    for i in range(len(fgi)):
        fgi[i].well['LATITUDE'] = ls.HeaderItem(mnemonic='LAT', unit='DEG', value=float('{:.4f}'.format((2 * np.random.sample() - 1)*90)), descr='Latitude')
        fgi[i].well['LONGITUDE'] = ls.HeaderItem(mnemonic='LONG', unit='DEG', value=float('{:.4f}'.format((2 * np.random.sample() - 1)*180)), descr='Longitude')
        start_idx = int(len(fgi_df)*i/len(fgi))
        stop_idx = int(len(fgi_df)*(i+1)/len(fgi))
        fgi_df.loc[start_idx:stop_idx, 'LATITUDE'] = fgi[i].well['LAT'].value # Sets specific latitude for each well
        fgi_df.loc[start_idx:stop_idx, 'LONGITUDE'] = fgi[i].well['LONG'].value # Sets specific longitude for each well
    # Location-test:
    """if (-50.0<=fgi[0].well['LAT'].value<=-5.0 and 90<=fgi[0].well['LONG']<=180):
        fgi[0].well['CTRY'].value = 'AUSTRALIA'
    else:
        fgi[0].well['CTRY'].value = 'OUTSIDE AUSTRALIA'"""
    return

In [ ]:
lat_long(list_of_all_wells, multiple_wells_df)

## Maps

In [ ]:
import json

from ipywidgets import Dropdown

from bqplot import Lines, Figure, LinearScale, LogScale, DateScale, Axis

from ipyleaflet import Map, GeoJSON, WidgetControl, Marker, basemaps, LayersControl, LayerGroup

In [ ]:
m = Map(basemap=basemaps.Esri.NatGeoWorldMap, center=(50,0), zoom=2)
wellnames = multiple_wells_df['WELLNAME'].unique()

In [ ]:
well_name = wellnames[0] 

x_scale = LinearScale()
depth_scale = LinearScale(reverse=True)
x_data = multiple_wells_df.loc[multiple_wells_df['WELLNAME'] == well_name]['CALI'].values
depth = multiple_wells_df.loc[multiple_wells_df['WELLNAME'] == well_name]['DEPTH (m)'].values

lines = Lines(x=x_data, y=depth, scales={'x': x_scale, 'y': depth_scale}, colors=['blue'])

ax_x = Axis(label='Cali', scale=x_scale, num_ticks=5)
ax_y = Axis(label='Depth', scale=depth_scale, orientation='vertical', side='left')

figure = Figure(axes=[ax_x, ax_y], title=well_name, marks=[lines], animation_duration=500, background_style={'color':'red'},
                layout={'max_height': '400px', 'max_width': '200px'})
#figure

In [ ]:
colors = {'GR':'blue', 'RD':'orange', 'RM':'red', 'RS':'darkred', 'NPHI':'gold', 'RHOB':'purple', 'DT':'brown', 'CALI':'green', 'BS':'gray'}

In [ ]:
parameter = 'GR'

def update_fig(well_name, param):
    if (param == 'RS' or param == 'RM' or param == 'RD'):
        x_scale = LogScale()
        ax_x.scale = LogScale()
        lines.scales['x'] = LogScale()
    else:
        x_scale = LinearScale()
        ax_x.scale = LinearScale()
        lines.scales['x'] = LinearScale()
    lines.x = multiple_wells_df.loc[multiple_wells_df['WELLNAME'] == well_name][param].values
    lines.y = multiple_wells_df.loc[multiple_wells_df['WELLNAME'] == well_name]['DEPTH (m)'].values
    lines.colors = [colors[param]]
    ax_x.label = param
    figure.title = well_name

In [ ]:
# Create GeoJSON format for markers

def geo_markers(dataset, well_idx):
    long = dataset['LONGITUDE'].unique()[well_idx]
    lat = dataset['LATITUDE'].unique()[well_idx]
    geo = {
            "type": "Feature",
            "properties": {
                "well_name": dataset['WELLNAME'].unique()[well_idx]
            },
            "geometry": {
                "type": "Point",
                "coordinates": [long, lat]
            }
         }
    return geo, long, lat

def make_geojson(dataset, well_idx_start, well_idx_stop):
    geojson = {'features': [geo_markers(dataset, well_idx)[0] for well_idx in np.arange(well_idx_start, well_idx_stop)]}
    return geojson, dataset

In [ ]:
def grouped_well_layers(amount_of_groups):
    data = []
    for i in range(amount_of_groups):
        data.append(make_geojson(multiple_wells_df, int(len(wellnames)/amount_of_groups*i), 
                            int(len(wellnames)/amount_of_groups*(i+1))))
    geojson_layers = [GeoJSON(data=data[i][0], name='Group'+str(i+1)) for i in range(amount_of_groups)]
    return geojson_layers

In [ ]:
well_groups = grouped_well_layers(3) # Create 3 well-groups
[m.add_layer(well_groups[i]) for i in range(len(well_groups))]

control = LayersControl(position='topleft')
m.add_control(control)

### Interactive plotting

In [ ]:
widget_control1 = WidgetControl(widget=figure, position='bottomright')

m.add_control(widget_control1)

def click(event, feature, **kwargs): # Update figure when clicking on well
    global well_name
    
    well_name = feature['properties']['well_name']
    update_fig(well_name, parameter)
    
[well_groups[i].on_click(click) for i in range(len(well_groups))]

In [ ]:
dropdown = Dropdown(options=num_curves[0], value=parameter)

def click(handler):
    global parameter

    parameter = handler['new']
    update_fig(well_name, parameter)

dropdown.observe(click, names='value')

widget_control2 = WidgetControl(widget=dropdown, position='bottomleft')

m.add_control(widget_control2)

In [ ]:
m